# games info dataset

In [1]:
import requests, re, time
import pandas, csv
import pickle

Initial tests for extracting useful information from one page:

In [2]:
resp = requests.get('https://www.truetrophies.com/game/A-Boy-and-His-Blob')
resp.status_code

200

In [ ]:
page = resp.content.decode()
page

In [4]:
genres = re.findall('/genre/.{1,30}\">', page)
genres = list(set(genres))
genres = [s[7:-2] for s in genres]
genres

['Platformer', 'Puzzle']

In [5]:
developer = re.findall('/developer/.{1,30}\">', page)
developer = list(set(developer))
developer = [s[11:-2] for s in developer]
developer

['WayForward-Technologies', 'Abstraction-Games']

In [6]:
publisher = re.findall('/publisher/.{1,30}\">', page)
publisher = list(set(publisher))
publisher = [s[11:-2] for s in publisher]
publisher

['Ziggurat']

In [7]:
date = re.findall('/xbox-releases/.{1,25}\">', page)
date = date[0][15:-2]
date

'2016/January/19'

In [8]:
size = re.findall('<dt>Size:</dt><dd>.{1,10}</dd><dt>', page)
size = size[0][18:-9]
size

'4.42GB'

Read the initaial csv file and get the pages from the links included in the file:

In [9]:
df = pandas.read_csv('drive/MyDrive/games_info.csv')
links = df['Links']

In [ ]:
pages = []
for link in links:
  resp = requests.get(link)
  time.sleep(4)                    #stop running for 4 seconds here to avoid getting IP blocked by the site
  page = resp.content.decode()
  pages.append(page)

The following cells are for extracting the useful information from the pages and adding the extracted information as new columns into the dataframe:

In [ ]:
genres_all = []
for page in pages:
  genres = re.findall('/genre/.{1,30}\">', page)
  genres = list(set(genres))
  genres = [s[7:-2] for s in genres]
  genres_all.append(genres)

In [ ]:
i = 0
for genres in genres_all:
  s = ''
  for genre in genres:
    s = s + genre + '|'
  s = s[0:-1]
  df['genres'][i] = s
  i += 1

In [ ]:
developers_all = []
for page in pages:
  developers = re.findall('/developer/.{1,35}\">', page)
  developers = list(set(developers))
  developers = [s[11:-2] for s in developers]
  developers_all.append(developers)

In [ ]:
i = 0
for developer in developers_all:
  s = ''
  for dev in developer:
    s = s + dev + '|'
  s = s[0:-1]
  df['developer'][i] = s
  i += 1

In [ ]:
publishers_all = []
for page in pages:
  publisher = re.findall('/publisher/.{1,40}\">', page)
  publisher = list(set(publisher))
  publisher = [s[11:-2] for s in publisher]
  publishers_all.append(publisher)

In [ ]:
i = 0
for pubs in publishers_all:
  s = ''
  for p in pubs:
    s = s + p + '|'
  s = s[0:-1]
  df['publisher'][i] = s
  i += 1

In [ ]:
dates_all = []
for page in pages:
  date = re.findall('/xbox-releases/.{1,25}\">', page)
  if (len(date) != 0):
    date = date[0][15:-2]
  else:
    date = ''
  dates_all.append(date)
i = 0
for d in dates_all:
  df['release_date'][i] = d
  i += 1

In [ ]:
sizes_all = []
for page in pages:
  size = re.findall('<dt>Size:</dt><dd>.{1,10}</dd><dt>', page)
  if (len(size) != 0):
    size = size[0][18:-9]
  else:
    size = ''
  sizes_all.append(size)
i = 0
for s in sizes_all:
  df['size'][i] = s
  i += 1

In [10]:
df

,index,game,score,leaderbord,gamers,comp_perc,rating,url,min_comp_time,max_comp_time,Links,genres,developer,publisher,release_date,size
0,0,A Boy and His Blob,638,2.02,2194,16.5,3.2,https://www.truetrophies.com/game/A-Boy-and-Hi...,15,20,https://www.truetrophies.com/game/A-Boy-and-Hi...,Puzzle|Platformer,WayForward-Technologies|Abstraction-Games,Ziggurat,2016/January/19,4.42GB
1,1,A Hat in Time,1992,1.53,7062,35.9,4.2,https://www.truetrophies.com/game/A-Hat-in-Tim...,15,20,https://www.truetrophies.com/game/A-Hat-in-Time,Platformer,Gears-for-Breakfast,Humble-Games,2017/December/05,4.04GB
2,2,A Hero and a Garden,1364,1.01,503,97.6,5.0,https://www.truetrophies.com/game/A-Hero-and-a...,0,1,https://www.truetrophies.com/game/A-Hero-and-a...,Visual-Novel,npckc,Ratalaika-Games,2020/August/28,90MB
3,3,A Hero and a Garden (EU),1363,1.01,581,97.8,2.9,https://www.truetrophies.com/game/A-Hero-and-a...,0,1,https://www.truetrophies.com/game/A-Hero-and-a...,Visual-Novel,npckc,Ratalaika-Games,2020/August/28,90MB
4,4,A King's Tale: Final Fantasy XV,637,2.02,21914,14.1,3.3,https://www.truetrophies.com/game/A-Kings-Tale...,4,5,https://www.truetrophies.com/game/A-Kings-Tale...,Action-Adventure|Beat-em-up,Empty-Clip-Studios,Square-Enix,2016/November/29,2.05GB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,1579,36 Fragments of Midnight,1367,1.06,8472,82.3,2.5,https://www.truetrophies.com/game/36-Fragments...,0,1,https://www.truetrophies.com/game/36-Fragments...,Platformer,Petite-Games,Ratalaika-Games,2017/December/19,147MB
1580,1580,36 Fragments of Midnight (Asia),1335,1.03,2131,88.9,2.4,https://www.truetrophies.com/game/36-Fragments...,0,1,https://www.truetrophies.com/game/36-Fragments...,Platformer,Petite-Games,eastasiasoft,2018/May/02,175MB
1581,1581,36 Fragments of Midnight (EU),1382,1.07,12273,79.2,2.4,https://www.truetrophies.com/game/36-Fragments...,0,1,https://www.truetrophies.com/game/36-Fragments...,Platformer,Petite-Games,Ratalaika-Games,2018/January/05,151MB
1582,1582,428: Shibuya Scramble,1943,1.47,916,41.5,4.2,https://www.truetrophies.com/game/428-Shibuya-...,40,50,https://www.truetrophies.com/game/428-Shibuya-...,Visual-Novel,Abstraction-Games,Spike-Chunsoft,2018/September/04,4.42GB


check the number of entries that don't have information correctly added. These information need to be added manually:

In [ ]:
count = 0
for i in range(len(df['size'])):
  if (df['size'][i] == ''):
    count += 1
count

27

Save to dataframe to the original csv file:

In [ ]:
df.to_csv('games_info.csv', index=False)

# user ratings dataset

In [12]:
df = pandas.read_csv('drive/MyDrive/games_info.csv')

Manipulating the links from games info dataset to make it work with metacritic links

In [ ]:
links = df['Links']
links = links.values.tolist()
links = [s[34:] for s in links]
links = [s.lower() for s in links]
links = ['https://www.metacritic.com/game/playstation-4/'+s+'/user-reviews?sort-by=most-helpful&num_items=100' for s in links]

Get the pages we need from metacritic:

In [ ]:
pages = []
userAgent = {'User-agent': 'Mozilla/5.0'}
for link in links:
  response = requests.get(link, headers=userAgent)
  page = response.content.decode()
  pages.append(page)
  time.sleep(4)

Check for the number of invalid responses. These entries does not have the correct link for metacritcs, so we need to fix these links manually.

In [ ]:
invalids = []
for i in range(len(pages)):
  not_found = re.findall('404 Page Not Found', pages[i])
  if len(not_found) != 0:
    invalids.append(i)
len(invalids)

571

Save the pages for any use in the future:

In [ ]:
pickle.dump(pages, open('drive/MyDrive/pages.txt', 'wb'))

In [14]:
pages = pickle.load(open('drive/MyDrive/pages.txt', 'rb'))

Reading the manually collected links and identify the subset that should actually be fixed. Then get the pages from metacritic:

In [ ]:
df_i = pandas.read_csv('invalids.csv')

In [ ]:
invalids_all = df_i.values.tolist()
invalids_fix = []
for pair in invalids_all:
  if pair[1][:5] == 'https':
    pair[1] = pair[1] + '/user-reviews?sort-by=most-helpful&num_items=100'
    invalids_fix.append(pair)
len(invalids_fix)

216

In [ ]:
userAgent = {'User-agent': 'Mozilla/5.0'}
for pair in invalids_fix:
  response = requests.get(pair[1], headers=userAgent)
  page = response.content.decode()
  pages[pair[0]] = page
  time.sleep(4)

Some initial tests for parsing information from one page and get the number of games with no user ratings:

In [16]:
user_ratings = re.findall('<a href="/user/.{1,100}</a>\n.{1,200}</div>\n\n.{1,200}\n.{1,200}</div>\n.{1,100}\n.{1,200}\n.{1,200}\n.{1,100}">.{1,2}</div>\n\n', pages[1147])
user_ratings

['<a href="/user/mrvertigo89">mrvertigo89</a>\n                                                                                                                                </div>\n\n                                                                                    <div class="date">Mar 24, 2019</div>\n                                                                            </div>\n                                    \n                                    <div class="review_grade">\n                                                \n<div class="metascore_w user medium game mixed indiv">7</div>\n\n',
 '<a href="/user/StrangeAeons">StrangeAeons</a>\n                                                                                                                                </div>\n\n                                                                                    <div class="date">Mar 25, 2019</div>\n                                                                            </di

In [18]:
ignore = []
for i in range(len(pages)):
  user_ratings = re.findall('<a href="/user/.{1,100}</a>\n.{1,200}</div>\n\n.{1,200}\n.{1,200}</div>\n.{1,100}\n.{1,200}\n.{1,200}\n.{1,100}">.{1,2}</div>\n\n', pages[i])
  if len(user_ratings) == 0:
    ignore.append(i)
len(ignore)

632

In [19]:
count = 0
for page in pages:
  next_page = re.findall('rel="next" href=".{1,200}"><span class="text">', page)
  if len(next_page) != 0:
    count += 1
count

188

In [20]:
next_page = re.findall('rel="next" href=".{1,200}"><span class="text">', pages[1147])
next_page[0][17:-21]

'/game/playstation-4/sekiro-shadows-die-twice/user-reviews?sort-by=most-helpful&num_items=100&page=1'

Getting the information from all the pages:

In [ ]:
for i in range(len(pages)):
  user_ratings = re.findall('<a href="/user/.{1,100}</a>\n.{1,200}</div>\n\n.{1,200}\n.{1,200}</div>\n.{1,100}\n.{1,200}\n.{1,200}\n.{1,100}\">.{1,2}</div>\n\n', pages[i])
  for raw_text in user_ratings:
    user = re.findall('a href="/user/.{1,100}">', raw_text)
    user = user[0][14:-2]
    rating = re.findall('">.{1,2}</div>\n\n', raw_text)
    rating = rating[0][2:-8]
    ratings.append((i, user, rating))
  next_page = re.findall('rel="next" href=".{1,200}"><span class="text">', pages[i])
  while len(next_page) != 0:
    next_page = next_page[0][17:-21]
    response = requests.get('https://www.metacritic.com' + next_page, headers=userAgent)
    page = response.content.decode()
    user_ratings = re.findall('<a href="/user/.{1,100}</a>\n.{1,200}</div>\n\n.{1,200}\n.{1,200}</div>\n.{1,100}\n.{1,200}\n.{1,200}\n.{1,100}\">.{1,2}</div>\n\n', page)
    time.sleep(4)
    for raw_text in user_ratings:
      user = re.findall('a href="/user/.{1,100}">', raw_text)
      user = user[0][14:-2]
      rating = re.findall('">.{1,2}</div>\n\n', raw_text)
      rating = rating[0][2:-8]
      ratings.append((i, user, rating))
    next_page = re.findall('rel="next" href=".{1,200}"><span class="text">', page)
len(ratings)

97529

Creating dataframe from the collected information and save it to a csv file:

In [ ]:
df_r = pandas.DataFrame(ratings, columns=['gameid', 'username', 'rating'])
df_r

,gameid,username,rating
0,0,consolfreak1982,8
1,0,GreenMachineG,6
2,1,bulger_paul,10
3,1,SuperDuck23,9
4,1,Wander22,8
...,...,...,...
97526,1577,SuperkenGaming,4
97527,1577,gobi123,10
97528,1582,Xtord,10
97529,1582,Peaceduke,5


In [ ]:
df_r.to_csv('user_ratings.csv', index=False)